In [1]:
import glob
import os
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.layers as K
from tqdm import tqdm
import cv2
import numpy as np

In [2]:
root= '/kaggle/input/lgg-mri-segmentation/kaggle_3m'
image=[]
label=[]
for d in os.listdir(root):
    path=os.path.join(root,d)
    if os.path.isdir(path):
            iters = int(len(os.listdir(path))/2)
            for i in range(iters): 
                file = os.path.join(path, d) + '_' +str(i+1) + '.tif'
                mask = os.path.join(path, d) + '_' +str(i+1) + '_mask.tif'
                image.append(file)
                label.append(mask)

In [3]:
def read_image(path):
    
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x,(128,128))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x,(128,128))
    x = x/255.0
    x = x > 0.5
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

def read_images(paths, tipefile):
    images = []
    
    if tipefile == 'images':
        for f in tqdm(paths):
            img = read_image(f)
            images.append(img)    
    else:
        
        for f in tqdm(paths):
            img = read_mask(f)

            images.append(img)
    
    imgs1 = np.array(images)
    
    return imgs1
images = read_images(image, 'images')
masks = read_images(label, 'masks')

100%|██████████| 3929/3929 [00:30<00:00, 130.29it/s]


In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt


def double_conv_block(x, n_filters):
    # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
    # Conv2D then ReLU activation
    x = layers.Conv2D(n_filters, 3, padding = "same", activation = "relu", kernel_initializer = "he_normal")(x)
    return x

def downsample_block(x, n_filters):
    f = double_conv_block(x, n_filters)
    p = layers.MaxPool2D(2)(f)
    p = layers.Dropout(0.3)(p)
    return f, p


def upsample_block(x, conv_features, n_filters):
    # upsample
    x = layers.Conv2DTranspose(n_filters, 3, 2, padding="same")(x)
    # concatenate
    x = layers.concatenate([x, conv_features])
    # dropout
    x = layers.Dropout(0.3)(x)
    # Conv2D twice with ReLU activation
    x = double_conv_block(x, n_filters)
    return x

def build_unet_model():
    # inputs
    inputs = layers.Input(shape=(128,128,3))

    # encoder: contracting path - downsample
    # 1 - downsample
    f1, p1 = downsample_block(inputs, 64)
    # 2 - downsample
    f2, p2 = downsample_block(p1, 128)
    # 3 - downsample
    f3, p3 = downsample_block(p2, 256)
    # 4 - downsample
    f4, p4 = downsample_block(p3, 512)

    # 5 - bottleneck
    bottleneck = double_conv_block(p4, 1024)

    # decoder: expanding path - upsample
    # 6 - upsample
    u6 = upsample_block(bottleneck, f4, 512)
    # 7 - upsample
    u7 = upsample_block(u6, f3, 256)
    # 8 - upsample
    u8 = upsample_block(u7, f2, 128)
    # 9 - upsample
    u9 = upsample_block(u8, f1, 64)

    # outputs
    outputs = layers.Conv2D(3, 1, padding="same", activation = "softmax")(u9)

    # unet model with Keras Functional API
    unet_model = tf.keras.Model(inputs, outputs, name="U-Net")

    return unet_model



unet_model = build_unet_model()

2023-01-05 13:50:18.866793: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 13:50:18.867722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 13:50:19.088094: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 13:50:19.088969: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-05 13:50:19.089809: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(images, 
                                                    masks, 
                                                    test_size=.25, 
                                                    random_state = 288)

In [7]:
print('X_train: ',  X_train.shape)
print('X_test: ',  X_test.shape)
print('y_train: ',  y_train.shape)
print('y_test: ',  y_test.shape)

X_train:  (2946, 128, 128, 3)
X_test:  (983, 128, 128, 3)
y_train:  (2946, 128, 128, 1)
y_test:  (983, 128, 128, 1)


In [9]:
unet_model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss="sparse_categorical_crossentropy",
                  metrics="accuracy")



BATCH_SIZE = 128
NUM_EPOCHS = 20



model_history = unet_model.fit(X_train,
                               y_train,
                               verbose=1,
                               epochs = NUM_EPOCHS,
                               validation_data=(X_test, y_test))

2023-01-05 13:59:57.662081: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2023-01-05 14:00:00.561796: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


93/93 [==============================] - 78s 611ms/step - loss: 0.3537 - accuracy: 0.9779 - val_loss: 0.0320 - val_accuracy: 0.9895
Epoch 2/20
93/93 [==============================] - 47s 506ms/step - loss: 0.0279 - accuracy: 0.9911 - val_loss: 0.0447 - val_accuracy: 0.9913
Epoch 3/20
93/93 [==============================] - 48s 512ms/step - loss: 0.0255 - accuracy: 0.9926 - val_loss: 0.0290 - val_accuracy: 0.9918
Epoch 4/20
93/93 [==============================] - 47s 507ms/step - loss: 0.0241 - accuracy: 0.9928 - val_loss: 0.0252 - val_accuracy: 0.9931
Epoch 5/20
93/93 [==============================] - 47s 508ms/step - loss: 0.0202 - accuracy: 0.9937 - val_loss: 0.0271 - val_accuracy: 0.9933
Epoch 6/20
93/93 [==============================] - 47s 508ms/step - loss: 0.0193 - accuracy: 0.9940 - val_loss: 0.0201 - val_accuracy: 0.9943
Epoch 7/20
93/93 [==============================] - 47s 508ms/step - loss: 0.0193 - accuracy: 0.9940 - val_loss: 0.0200 - val_accuracy: 0.9942
Epoch 8/20

In [10]:
from tensorflow.keras.models import load_model

unet_model.save('segment.h5')